In [ ]:
#!/usr/bin/env python

import os
from glob import glob

# Import new things that we'll need
import pandas as pd
import numpy as np
import nipype.interfaces.afni as afni
import nipype.interfaces.fsl as fsl
from nipype.interfaces.utility import Function
import seaborn as sns
import nibabel as nb
import json
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe 
import matplotlib.pyplot as plt

sid = ['021']
base_dir = '/home/mrive301/Documents/CogNeuro_Methods/CogNeuro_proj'
work_dir = '/scratch/madlab/mvrn_cogneuro'

#Creating a function to iterate over all tasks in functional directory
def task_iterate(func_scan):
    files = os.path.join(base_dir, f'dset/sub-{sid[0]}/{func_scan}')
    for file in files:
        get_loc_json = sorted(glob(files + '/*localizer*.json')) 
        get_loc_nii = sorted(glob(files + '/*localizer*.nii.gz'))
        get_task_json = sorted(glob(files + '/*task-*.json'))
        get_task_nii = sorted(glob(files + '/*task-*.nii.gz'))
        #I could add other tasks to this list, if the func directory had more
        return get_loc_json, get_loc_nii, get_task_json, get_task_nii

#Returning all of the functional files
functional = task_iterate("func")

#Getting the functional json files
study_func_json = functional[2:3]

#Returning the functional nifty files
study_func_files = functional[3:]

'''
========================================================
Creating function to eliminate mapnode directory structure 
and saving output func_filed in a dingle directory
========================================================
'''
def get_subs(func_files):
    '''Produces Name Substitutions for Each Contrast'''
    subs = []
    for curr_run in range(len(func_files)):
        subs.append(('_tshifter%d' %curr_run, ''))
        subs.append(('_volreg%d' %curr_run, ''))
    return subs

'''
========================================================
Receiving text file with number of outliers per volume. Then,
we find which volume index has the least number of outliers.

We search over the first 201 volumes.

If index returns list, that's becaus there were multiple
volumes wih the same number of outliers. If so, we pick the 
first one.
========================================================
'''

def best_vol(outlier_count):
    #Searching over 201 volumes which one has the least number of 
        #outliers. If best one found, we return such one. 
    best_vol_num = outlier_count.index(min(outlier_count[:200]))
    #If not, and all have same number of outliers, we return first one
    if isinstance(best_vol_num, list):
        best_vol_num = best_vol_num[0]
    return best_vol_num

'''
========================================================
Creating list with ST for each study run
========================================================
'''
slice_timing_list = []
for curr_json in study_func_json:
    curr_json_data = open(curr_json) #opening json with func data
    curr_study_func_metadata = json.load(curr_json_data) #loading json files
    #Appending functional json ST files
    slice_timing_list.append(curr_study_func_metadata['SliceTiming'])

# Establishing a nipype work flow that will eventually be executed
cogneuro_wf = pe.Workflow(name='cogneuro_wf')

'''
========================================================
The following contributes to the function above that 
eliminates the mapnode by substituting the "func_files" 
name with "subs". 
========================================================
'''
# Create a Function node to substitute names of files created during pipeline
#Based on first function created int this cell
getsubs = pe.Node(Function(input_names=['func_files'],
                                             output_names=['subs'],
                                             function=get_subs),
                              name='getsubs')
getsubs.inputs.func_files = study_func_files

'''
========================================================
Using 3dToutcount to find the number of outliers per vol.
This is allowing us to create our basis for motion correction
by selecting the first volume with the lowest outlier number.
========================================================
'''
id_outliers = pe.Node(afni.OutlierCount(),
                      name = 'id_outliers')
#Trying another volume...closer to where the functional
#localizer to see what would happen...
id_outliers.inputs.in_file = study_func_files[7]
id_outliers.inputs.automask = True
id_outliers.inputs.out_file = 'outlier_file'

'''
========================================================
Passig the node that will allow us to get the best volume (the
one with the min number of outliers). This is based on 
previous outlier function above.
========================================================
'''
getbestvol = pe.Node(Function(input_names=['outlier_count'],
                                  output_names=['best_vol_num'],
                                  function=best_vol),
                                   name='getbestvol')
cogneuro_wf.connect(id_outliers, 'out_file', getbestvol, 'outlier_count')

'''
=========================================================
Extracting middle-ish volume of the first run as our reference
image.
=========================================================
'''
extractref = pe.Node(fsl.ExtractROI(t_size=1),
                     name = "extractref")
extractref.inputs.in_file = study_func_files[7]
#extractref.inputs.t_min = int(np.ceil(nb.load(study_func_files[0]).shape[3]/2)) #PICKING MIDDLE
cogneuro_wf.connect(getbestvol, 'best_vol_num', extractref, 't_min')

'''
=============================================================
Aaron's question: Would I want to do slice timing correction here?
Why? Why not?
Answer: No, you want to do so after motion correction, otherwise 
you may have motion intensity differences propagated across time.
=============================================================
'''

'''
=============================================================
Running AFNI's 3dvolreg for motion correction. We iterate over
our functional files which we pass functional data from the
slice timing correction node before we use the earliest volume with
the least number of outliers during the first run
as the base file to register to.
=============================================================
'''
'''
volreg = pe.MapNode(afni.Volreg(),
                    iterfield=['in_file'],
                    name = 'volreg')
volreg.inputs.outputtype = 'NIFTI_GZ'
volreg.inputs.zpad = 4
cogneuro_wf.connect(tshifter, 'out_file', volreg, 'in_file')
cogneuro_wf.connect(extractref, 'roi_file', volreg, 'basefile')

# Below is the code if I wanted to use the FSL McFlirt command
'''
motion_corr = pe.MapNode(fsl.MCFLIRT(),
                         iterfield=['in_file'],
                         name = 'motion_corr')
motion_corr.inputs.output_type = 'NIFTI_GZ'

'''
=============================================================
Running AFNIs 3dTshift to do ST correction. We input the 
functional files as a list, along with the ST as a list of lists. We
use a mapnode to iterate over the two so we can run them
at the same time on the HPC.
=============================================================
'''

tshifter = pe.MapNode(afni.TShift(),
                      iterfield=['in_file','slice_timing'],
                      name = 'tshifter')
tshifter.inputs.in_file = study_func_files
tshifter.inputs.tr = str(curr_study_func_metadata['RepetitionTime'])
tshifter.inputs.slice_timing = slice_timing_list
#Reversing slice timing
tshift.inputs.slice_encoding_direction = 'k-'
#Changing interpolation to linear (default = Fourier)
tshift.inputs.interp = 'linear'
tshifter.inputs.outputtype = 'NIFTI_GZ'
cogneuro_wf.connect(motion_corr, 'out_file', tshifter, 'in_file')
cogneuro_wf.connect(extractref, 'roi_file', tshifter, 'ref_file')

'''
=============================================================
Sinking data!
=============================================================
'''
datasink = pe.Node(nio.DataSink(), name="datasink")
datasink.inputs.base_directory = os.path.join(base_dir, 'derivatives')
datasink.inputs.container = f'sub-{sid[0]}'
cogneuro_wf.connect(tshifter, 'out_file', datasink, 'sltime_corr')
cogneuro_wf.connect(extractref, 'roi_file', datasink, 'study_ref')
cogneuro_wf.connect(volreg, 'out_file', datasink, 'motion.@corrfile')
cogneuro_wf.connect(volreg, 'oned_matrix_save', datasink, 'motion.@matrix')
cogneuro_wf.connect(volreg, 'oned_file', datasink, 'motion.@par')
cogneuro_wf.connect(getsubs, 'subs', datasink, 'substitutions')

# The following two lines set a work directory outside of my 
# local git repo and runs the workflow
cogneuro_wf.base_dir = work_dir + f'/cogneuroworkdir_v2/sub-{sid[0]}'
cogneuro_wf.run()